<font size="6px" font color="blue" face="Times New Roman"><b>Modelo de hipótese para dados de Well Logging</b></font>


### Script desenvolvido por Victor Carreira <br>
### Colaboração de Rodrigos Mota e Bijani

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Pacotes
#% matplotlib inline
from sys import maxsize
import numpy as np
from numpy import set_printoptions
import pylab as py
import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib import style
import random
from random import shuffle
import scipy as sp
from scipy import ndimage
from scipy import misc
from PIL import Image, ImageFilter
from IPython.display import Latex
#----------------------------------
#novos pacotes de imagem
import imageio
from skimage import data, io, filters
from skimage import img_as_float
from skimage.transform import radon, rescale
from skimage.morphology import reconstruction

In [ ]:
#antigo método de leitura de rasters
#ma = img.imread('amazonas.png') #carrega o modelo de hipótese H1 e transforma o raster em uma matriz de dados. Função imread()
#ma = misc.imread('amazonas.png',mode='P')
#ma = imageio.imread('amazonas.png')


#print(ma.shape) #mostra a matriz

#print(ma[:50,:50,1])
# no 'mode' usar:
#* 'L' (8-bit pixels, black and white)
#* 'P' (8-bit pixels, mapped to any other mode using a color palette)
#* 'RGB' (3x8-bit pixels, true color)
#* 'RGBA' (4x8-bit pixels, true color with transparency mask)
#* 'CMYK' (4x8-bit pixels, color separation)
#* 'YCbCr' (3x8-bit pixels, color video format)
#* 'I' (32-bit signed integer pixels)
#* 'F' (32-bit floating point pixels)

#plt.matshow(ma,vmin=0.0,vmax=4.0)#Cria o gráfico com a matriz da imagem (cria um valor mínimo e máximo)
#plt.colorbar()#Define o tipo de cor da barra de dados
#plt.show()#mostra o gráfico

In [ ]:
ma = io.imread("../figs/amazonas.png", as_gray=False)
plt.style.use(['classic'])
py.rcParams['figure.figsize'] = (20.0, 30.0)#new figure dimension
plt.title("Modelo")
plt.imshow(ma)
plt.grid()
plt.show()


#np.set_printoptions(threshold=maxsize)
#np.set_printoptions(threshold=np.nan)
#np.set_printoptions(threshold=np.inf)
#print(np.array2string(ma, threshold=np.nan, max_line_width=np.nan))

In [ ]:
# Visualizacao da matriz de dados:
print(np.shape(ma))

In [ ]:
############################################### PERFILAGEM SINTETICA ##########################################
mb = ma[:,250]#Faz uma amostragem vertical na seção. Essa amostragem simula o poço. Poço 1.
#print('mb', np.array2string(mb, threshold=np.nan, max_line_width=np.nan))#imprime o RGBA do poço teste
#py.rcParams['figure.figsize'] = (4.0, 15.0)#new figure dimension
#plt.grid() # bota a gridagem
#plt.gca().invert_yaxis() # inverte o eixo y
#plt.imshow(mb,cmap='hsv')
#plt.show()

In [ ]:
print(mb)

In [ ]:
############# CRIACAO DO PERFIL DE DENSIDADE #############

# ---- densidades das rochas -------: 
# 0 = folhelho | 2.4  |
# 1 = A2       | 2.1  |
# 2 = A1       | 2.05 |
# 3 = marga    | 2.2  |
# 4 = Basalto  | 2.8  |

RHOmb = [0.0] * len(mb)
prof = [] #cria uma lista vazia

for i in range(len(mb)): #Cria um Looping com índice i que varia de 0 até mb-1. Ou seja, 999 elementos.
    if mb[i].all() == 0: #declara que quando o índice i for igual a 0 na posição mn ...
        RHOmb[i] = 2.4   #o elemento do vetor assumirá o valor de 2.4...e assim sucessivamente ... 
    if mb[i].all() == 1:
        RHOmb[i] = 2.1
    if mb[i].all() == 2:
        RHOmb[i] = 2.05
    if mb[i].all() == 3:
        RHOmb[i] = 2.2
    if mb[i].all() == 4:
        RHOmb[i] = 2.6
    prof.append(i)# Preenche o vetor prof com os valores do índice i. Comando append()

In [ ]:
## Adiconando ruído randômico gaussiano normal
pure = np.linspace(-0.1, 0.1, len(RHOmb)) # Cria um vetor para receber a imagem da função densidade de probabilidade
noise = np.random.normal(0, 0.06, pure.shape) #Vetor ruído
signal_RHOmb = RHOmb + noise # sinal original com ruído

In [ ]:
####### Plot do perfil de densidade ###############
plt.plot(signal_RHOmb,prof, 'r') 
plt.style.use(['classic'])
plt.xlabel('Densidade ($g/cm^3$)', fontsize='small')
plt.ylabel('Profundidade ($a definir$)', fontsize='small')
py.rcParams['figure.figsize'] = (10.0, 6.0)#new figure dimension
plt.title('$Po\c{c}o$ mb', fontsize='xx-large')
plt.grid() # bota a gridagem
plt.gca().invert_yaxis() # inverte o eixo y
plt.show()

In [ ]:
##############################################  RAIO GAMA ################################################

#              Raio-Gama
# 0 = Folhelho | 120.00  | 
# 1 = A2       | 10.00   |
# 2 = A1       | 4.80    |
# 3 = marga    | 7.20    |
# 4 = Basalto  | 1.20    |

GRmb = [0.0]*len(mb)
prof = []

for i in range(len(mb)):
    if mb[i].all() == 0:
        GRmb[i] = 120.00
    if mb[i].all() == 1:
        GRmb[i] = 10.00
    if mb[i].all() == 2:
        GRmb[i] = 4.80
    if mb[i].all() == 3:
        GRmb[i] = 7.20
    if mb[i].all() == 4:
        GRmb[i] = 1.20
    prof.append(i)

In [ ]:
## Adiconando ruído randômico gaussiano normal
pure = np.linspace(-0.1, 0.1, len(GRmb)) # Cria um vetor para receber a imagem da função densidade de probabilidade
noise = np.random.normal(0, 7, pure.shape) #Vetor ruído
signal_GRmb = GRmb + noise # sinal original com ruído

In [ ]:
# Plot do perfil de raio gama ############
plt.plot(signal_GRmb,prof, 'b')
plt.style.use(['classic'])
plt.xlabel('Raio-Gama')
plt.ylabel('Profundidade')
plt.title('$Po\c{c}o$ mb', fontsize='xx-large')
plt.grid()
plt.gca().invert_yaxis()
plt.show()

## TODO: Atividades a serem desenvolvidas para aprimorar o script: <br>
1) Converter as coordenadas da bacia a fim de facilitar a escolha dos poços; <br> 
2) Padronizar a escolha dos poços (interativamente);<br>
3) Incluir mais propriedades fisicas (perfil sônico, perfil porosidade, etc); <br>
4) Aumentar a quantidade de poços sintéticos;